In [1]:
!pip install -q mujoco wandb mujoco_mjx ml_collections brax gdown

In [2]:
#@title Import packages for plotting and creating graphics
import time
import itertools
import numpy as np
from typing import Callable, NamedTuple, Optional, Union, List

# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

Installing mediapy:


In [3]:
#@title Import MuJoCo, MJX, and Brax
from datetime import datetime
from etils import epath
import functools
from IPython.display import HTML
from typing import Any, Dict, Sequence, Tuple, Union
import os
from ml_collections import config_dict


import jax
from jax import numpy as jp
import numpy as np
from flax.training import orbax_utils
from flax import struct
from matplotlib import pyplot as plt
import mediapy as media
from orbax import checkpoint as ocp

import mujoco
from mujoco import mjx

from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.envs.base import Env, PipelineEnv, State
from brax.mjx.base import State as MjxState
from brax.training.agents.ppo import train as ppo
from brax.training.agents.ppo import networks as ppo_networks
from brax.io import html, mjcf, model

import wandb

#login to wandb

In [4]:
api_token = '50c8814867c630e4f1649261257a9e728b7cebb6'
try:
    if wandb.login(key=api_token, relogin=False):
        print("Successfully logged in to Weights & Biases.")
    else:
        print("Login failed. Please check your API token.")
except Exception as e:
    print(f"An error occurred: {e}")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Successfully logged in to Weights & Biases.


In [5]:
api = wandb.Api()
run = api.run(f"tmptmp-tmp/tmp/runs/frlq4o5k")
run.file('tmp/humanoid/ckpts/227082240.zip').download(replace=True)

<_io.TextIOWrapper name='./tmp/humanoid/ckpts/227082240.zip' mode='r' encoding='UTF-8'>

In [6]:
import zipfile
import os

def unzip_file(zip_path, extract_to='.'):
    """
    Unzips a zip file to the specified directory and returns the path to the extracted directory.

    Parameters:
    zip_path (str): The path to the zip file.
    extract_to (str): The directory to extract the files to (default is the current directory).

    Returns:
    str: The path to the directory where the files were extracted.
    """
    # Ensure the extraction directory exists
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)

    # Unzip the file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

    # Return the path to the unzipped directory
    unzipped_dir = os.path.join(extract_to, os.path.splitext(os.path.basename(zip_path))[0])
    return unzipped_dir

In [7]:
unzip_file('/kaggle/working/tmp/humanoid/ckpts/227082240.zip')

'./227082240'

# Experiment Config

In [8]:
config={
    "algorithm": "PPO",
    "Policy Network Architecture": "default",
    "Value Network Architecture": "default",
    "Obs":"data.qpos[2:]+data.qvel[3:6]+data.qacc[0:2]",
    "reward":"multi-stage",
    "num_timesteps":100_000_000,
    "num_evals":10,
    "reward_scaling":1,
    "episode_length":1000,
    "normalize_observations": True,
    "action_repeat": 1,
    "unroll_length": 10,
    "num_minibatches": 32,
    "num_updates_per_batch": 8,
    "discounting": 0.97,
    "learning_rate": 3e-4,
    "entropy_cost": 1e-3,
    "num_envs": 2048,
    "batch_size": 1024,
    "seed": 0,
    "xml_path": None,
    "restore_checkpoint_path": "/kaggle/working/227082240",
    "create_checkpoint_path": None
}

# Utilities

In [9]:
# a utility function to create a subset from a given config
sub_config = lambda config, wanted_keys: dict((k, config[k]) for k in wanted_keys if k in config)

# Train

In [10]:
#!gdown 1gM7QuhJCsGP_U7iROW8QTg0lOg7cCSYj

In [11]:
xml_str="""<!-- Copyright 2021 DeepMind Technologies Limited

     Licensed under the Apache License, Version 2.0 (the "License");
     you may not use this file except in compliance with the License.
     You may obtain a copy of the License at

         http://www.apache.org/licenses/LICENSE-2.0

     Unless required by applicable law or agreed to in writing, software
     distributed under the License is distributed on an "AS IS" BASIS,
     WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
     See the License for the specific language governing permissions and
     limitations under the License.
-->

<mujoco model="Humanoid">
  <option timestep="0.005" iterations="1" ls_iterations="4">
    <flag eulerdamp="disable"/>
  </option>

  <visual>
    <map force="0.1" zfar="30"/>
    <rgba haze="0.15 0.25 0.35 1"/>
    <global offwidth="2560" offheight="1440" elevation="-20" azimuth="120"/>
  </visual>

  <statistic center="0 0 0.7"/>

  <asset>
    <texture type="skybox" builtin="gradient" rgb1=".3 .5 .7" rgb2="0 0 0" width="32" height="512"/>
    <texture name="body" type="cube" builtin="flat" mark="cross" width="128" height="128" rgb1="0.8 0.6 0.4" rgb2="0.8 0.6 0.4" markrgb="1 1 1" random="0.01"/>
    <material name="body" texture="body" texuniform="true" rgba="0.8 0.6 .4 1"/>
    <texture type="skybox" builtin="gradient" rgb1="0.3 0.5 0.7" rgb2="0 0 0" width="512" height="3072"/>
    <!--texture type="2d" name="groundplane" builtin="checker" mark="edge" rgb1="0.2 0.3 0.4" rgb2="0.1 0.2 0.3"
      markrgb="0.8 0.8 0.8" width="512" height="512"/>
    <material name="groundplane" texture="groundplane" texuniform="true" texrepeat="1 1" reflectance="0.2"/-->
  </asset>

  <default>
    <motor ctrlrange="-1 1" ctrllimited="true"/>
    <default class="body">

      <!-- geoms -->
      <!-- TODO(robotics-simulation): support condim=1 for humanoid capsules. -->
      <geom type="capsule" condim="3" friction=".7" solimp=".9 .99 .003" solref=".015 1" material="body" contype="0" conaffinity="0"/>
      <default class="thigh">
        <geom size=".06"/>
      </default>
      <default class="shin">
        <geom fromto="0 0 0 0 0 -.3"  size=".049"/>
      </default>
      <default class="foot">
        <geom size=".027"/>
        <default class="foot1">
          <geom fromto="-.07 -.01 0 .14 -.03 0"/>
        </default>
        <default class="foot2">
          <geom fromto="-.07 .01 0 .14  .03 0"/>
        </default>
      </default>
      <default class="arm_upper">
        <geom size=".04"/>
      </default>
      <default class="arm_lower">
        <geom size=".031"/>
      </default>
      <default class="hand">
        <geom type="sphere" size=".04"/>
      </default>

      <!-- joints -->
      <joint type="hinge" damping=".2" stiffness="1" armature=".01" limited="true" solimplimit="0 .99 .01"/>
      <default class="joint_big">
        <joint damping="5" stiffness="10"/>
        <default class="hip_x">
          <joint range="-30 10"/>
        </default>
        <default class="hip_z">
          <joint range="-60 35"/>
        </default>
        <default class="hip_y">
          <joint axis="0 1 0" range="-150 20"/>
        </default>
        <default class="joint_big_stiff">
          <joint stiffness="20"/>
        </default>
      </default>
      <default class="knee">
        <joint pos="0 0 .02" axis="0 -1 0" range="-160 2"/>
      </default>
      <default class="ankle">
        <joint range="-50 50"/>
        <default class="ankle_y">
          <joint pos="0 0 .08" axis="0 1 0" stiffness="6"/>
        </default>
        <default class="ankle_x">
          <joint pos="0 0 .04" stiffness="3"/>
        </default>
      </default>
      <default class="shoulder">
        <joint range="-85 60"/>
      </default>
      <default class="elbow">
        <joint range="-100 50" stiffness="0"/>
      </default>
    </default>
  </default>

  <worldbody>
    <!--light directional="true" diffuse=".8 .8 .8" pos="0 0 10" dir="0 0 -10"/>
    <geom name="floor" type="hfield" hfield="hfield" conaffinity="1" pos="0 0 0" material="stairs_hfield"/-->
    <body name="stairs" pos="-0.5 0 -0.5">
        <!-- 20 steps, each 18 cm high, 10 meters wide, and spread evenly across 1000 meters -->
        <geom name="step1" type="box" pos="5 0 0.18" size="5 10 0.18" rgba="0.1 0.1 0.5 1" conaffinity="3" />  <!-- Dark Blue -->
        <geom name="step2" type="box" pos="10 0 0.36" size="5 10 0.18" rgba="0.2 0.2 0.6 1" conaffinity="3"/>  <!-- Lighter Blue -->
        <geom name="step3" type="box" pos="20 0 0.54" size="5 10 0.18" rgba="0.1 0.1 0.5 1" conaffinity="3"/> <!-- Dark Blue -->
        <geom name="step4" type="box" pos="30 0 0.72" size="5 10 0.18" rgba="0.2 0.2 0.6 1" conaffinity="3"/> <!-- Lighter Blue -->
        <geom name="step5" type="box" pos="40 0 0.90" size="5 10 0.18" rgba="0.1 0.1 0.5 1" conaffinity="3"/> <!-- Dark Blue -->
        <geom name="step6" type="box" pos="50 0 1.08" size="5 10 0.18" rgba="0.2 0.2 0.6 1" conaffinity="3"/> <!-- Lighter Blue -->
        <geom name="step7" type="box" pos="60 0 1.26" size="5 10 0.18" rgba="0.1 0.1 0.5 1" conaffinity="3"/> <!-- Dark Blue -->
        <geom name="step8" type="box" pos="70 0 1.44" size="5 10 0.18" rgba="0.2 0.2 0.6 1" conaffinity="3"/> <!-- Lighter Blue -->
        <geom name="step9" type="box" pos="80 0 1.62" size="5 10 0.18" rgba="0.1 0.1 0.5 1" conaffinity="3"/> <!-- Dark Blue -->
        <geom name="step10" type="box" pos="90 0 1.80" size="5 10 0.18" rgba="0.2 0.2 0.6 1" conaffinity="3"/> <!-- Lighter Blue -->
    </body>

    <light name="spotlight" mode="targetbodycom" target="torso" diffuse=".8 .8 .8" specular="0.3 0.3 0.3" pos="0 -6 4" cutoff="30"/>
    <body name="torso" pos="0 0 1.282" childclass="body">
      <light name="top" pos="0 0 2" mode="trackcom"/>
      <camera name="back" pos="-3 0 1" xyaxes="0 -1 0 1 0 2" mode="trackcom"/>
      <camera name="side" pos="0 -3 1" xyaxes="1 0 0 0 1 2" mode="trackcom"/>
      <freejoint name="root"/>
      <geom name="torso" fromto="0 -.07 0 0 .07 0" size=".07"/>
      <geom name="waist_upper" fromto="-.01 -.06 -.12 -.01 .06 -.12" size=".06"/>
      <body name="head" pos="0 0 .19">
        <geom name="head" type="sphere" size=".09"/>
        <camera name="egocentric" pos=".09 0 0" xyaxes="0 -1 0 .1 0 1" fovy="80"/>
      </body>
      <body name="waist_lower" pos="-.01 0 -.26">
        <geom name="waist_lower" fromto="0 -.06 0 0 .06 0" size=".06"/>
        <joint name="abdomen_z" pos="0 0 .065" axis="0 0 1" range="-45 45" class="joint_big_stiff"/>
        <joint name="abdomen_y" pos="0 0 .065" axis="0 1 0" range="-75 30" class="joint_big"/>
        <body name="pelvis" pos="0 0 -.165">
          <joint name="abdomen_x" pos="0 0 .1" axis="1 0 0" range="-35 35" class="joint_big"/>
          <geom name="butt" fromto="-.02 -.07 0 -.02 .07 0" size=".09"/>
          <body name="thigh_right" pos="0 -.1 -.04">
            <joint name="hip_x_right" axis="1 0 0" class="hip_x"/>
            <joint name="hip_z_right" axis="0 0 1" class="hip_z"/>
            <joint name="hip_y_right" class="hip_y"/>
            <geom name="thigh_right" fromto="0 0 0 0 .01 -.34" class="thigh"/>
            <body name="shin_right" pos="0 .01 -.4">
              <joint name="knee_right" class="knee"/>
              <geom name="shin_right" class="shin"/>
              <body name="foot_right" pos="0 0 -.39">
                <joint name="ankle_y_right" class="ankle_y"/>
                <joint name="ankle_x_right" class="ankle_x" axis="1 0 .5"/>
                <geom name="foot1_right" class="foot1"/>
                <geom name="foot2_right" class="foot2"/>
              </body>
            </body>
          </body>
          <body name="thigh_left" pos="0 .1 -.04">
            <joint name="hip_x_left" axis="-1 0 0" class="hip_x"/>
            <joint name="hip_z_left" axis="0 0 -1" class="hip_z"/>
            <joint name="hip_y_left" class="hip_y"/>
            <geom name="thigh_left" fromto="0 0 0 0 -.01 -.34" class="thigh"/>
            <body name="shin_left" pos="0 -.01 -.4">
              <joint name="knee_left" class="knee"/>
              <geom name="shin_left" fromto="0 0 0 0 0 -.3" class="shin"/>
              <body name="foot_left" pos="0 0 -.39">
                <joint name="ankle_y_left" class="ankle_y"/>
                <joint name="ankle_x_left" class="ankle_x" axis="-1 0 -.5"/>
                <geom name="foot1_left" class="foot1"/>
                <geom name="foot2_left" class="foot2"/>
              </body>
            </body>
          </body>
        </body>
      </body>
      <body name="upper_arm_right" pos="0 -.17 .06">
        <joint name="shoulder1_right" axis="2 1 1"  class="shoulder"/>
        <joint name="shoulder2_right" axis="0 -1 1" class="shoulder"/>
        <geom name="upper_arm_right" fromto="0 0 0 .16 -.16 -.16" class="arm_upper"/>
        <body name="lower_arm_right" pos=".18 -.18 -.18">
          <joint name="elbow_right" axis="0 -1 1" class="elbow"/>
          <geom name="lower_arm_right" fromto=".01 .01 .01 .17 .17 .17" class="arm_lower"/>
          <body name="hand_right" pos=".18 .18 .18">
            <geom name="hand_right" zaxis="1 1 1" class="hand"/>
          </body>
        </body>
      </body>
      <body name="upper_arm_left" pos="0 .17 .06">
        <joint name="shoulder1_left" axis="-2 1 -1" class="shoulder"/>
        <joint name="shoulder2_left" axis="0 -1 -1"  class="shoulder"/>
        <geom name="upper_arm_left" fromto="0 0 0 .16 .16 -.16" class="arm_upper"/>
        <body name="lower_arm_left" pos=".18 .18 -.18">
          <joint name="elbow_left" axis="0 -1 -1" class="elbow"/>
          <geom name="lower_arm_left" fromto=".01 -.01 .01 .17 -.17 .17" class="arm_lower"/>
          <body name="hand_left" pos=".18 -.18 .18">
            <geom name="hand_left" zaxis="1 -1 1" class="hand"/>
          </body>
        </body>
      </body>
    </body>
  </worldbody>

  <contact>
    <exclude body1="waist_lower" body2="thigh_right"/>
    <exclude body1="waist_lower" body2="thigh_left"/>
    <!-- Contact pairs for step1 -->
    <pair geom1="foot1_left" geom2="step1"/>
    <pair geom1="foot1_right" geom2="step1"/>
    <pair geom1="foot2_left" geom2="step1"/>
    <pair geom1="foot2_right" geom2="step1"/>

    <!-- Contact pairs for step2 -->
    <pair geom1="foot1_left" geom2="step2"/>
    <pair geom1="foot1_right" geom2="step2"/>
    <pair geom1="foot2_left" geom2="step2"/>
    <pair geom1="foot2_right" geom2="step2"/>

    <!-- Contact pairs for step3 -->
    <pair geom1="foot1_left" geom2="step3"/>
    <pair geom1="foot1_right" geom2="step3"/>
    <pair geom1="foot2_left" geom2="step3"/>
    <pair geom1="foot2_right" geom2="step3"/>

    <!-- Contact pairs for step4 -->
    <pair geom1="foot1_left" geom2="step4"/>
    <pair geom1="foot1_right" geom2="step4"/>
    <pair geom1="foot2_left" geom2="step4"/>
    <pair geom1="foot2_right" geom2="step4"/>

    <!-- Contact pairs for step5 -->
    <pair geom1="foot1_left" geom2="step5"/>
    <pair geom1="foot1_right" geom2="step5"/>
    <pair geom1="foot2_left" geom2="step5"/>
    <pair geom1="foot2_right" geom2="step5"/>

    <!-- Contact pairs for step6 -->
    <pair geom1="foot1_left" geom2="step6"/>
    <pair geom1="foot1_right" geom2="step6"/>
    <pair geom1="foot2_left" geom2="step6"/>
    <pair geom1="foot2_right" geom2="step6"/>

    <!-- Contact pairs for step7 -->
    <pair geom1="foot1_left" geom2="step7"/>
    <pair geom1="foot1_right" geom2="step7"/>
    <pair geom1="foot2_left" geom2="step7"/>
    <pair geom1="foot2_right" geom2="step7"/>

    <!-- Contact pairs for step8 -->
    <pair geom1="foot1_left" geom2="step8"/>
    <pair geom1="foot1_right" geom2="step8"/>
    <pair geom1="foot2_left" geom2="step8"/>
    <pair geom1="foot2_right" geom2="step8"/>

    <!-- Contact pairs for step9 -->
    <pair geom1="foot1_left" geom2="step9"/>
    <pair geom1="foot1_right" geom2="step9"/>
    <pair geom1="foot2_left" geom2="step9"/>
    <pair geom1="foot2_right" geom2="step9"/>

    <!-- Contact pairs for step10 -->
    <pair geom1="foot1_left" geom2="step10"/>
    <pair geom1="foot1_right" geom2="step10"/>
    <pair geom1="foot2_left" geom2="step10"/>
    <pair geom1="foot2_right" geom2="step10"/>


  </contact>

  <!-- <tendon>
    <fixed name="hamstring_right" limited="true" range="-0.3 2">
      <joint joint="hip_y_right" coef=".5"/>
      <joint joint="knee_right" coef="-.5"/>
    </fixed>
    <fixed name="hamstring_left" limited="true" range="-0.3 2">
      <joint joint="hip_y_left" coef=".5"/>
      <joint joint="knee_left" coef="-.5"/>
    </fixed>
  </tendon> -->

  <actuator>
    <motor name="abdomen_y"       gear="40"  joint="abdomen_y"/>
    <motor name="abdomen_z"       gear="40"  joint="abdomen_z"/>
    <motor name="abdomen_x"       gear="40"  joint="abdomen_x"/>
    <motor name="hip_x_right"     gear="40"  joint="hip_x_right"/>
    <motor name="hip_z_right"     gear="40"  joint="hip_z_right"/>
    <motor name="hip_y_right"     gear="120" joint="hip_y_right"/>
    <motor name="knee_right"      gear="80"  joint="knee_right"/>
    <motor name="ankle_x_right"   gear="20"  joint="ankle_x_right"/>
    <motor name="ankle_y_right"   gear="20"  joint="ankle_y_right"/>
    <motor name="hip_x_left"      gear="40"  joint="hip_x_left"/>
    <motor name="hip_z_left"      gear="40"  joint="hip_z_left"/>
    <motor name="hip_y_left"      gear="120" joint="hip_y_left"/>
    <motor name="knee_left"       gear="80"  joint="knee_left"/>
    <motor name="ankle_x_left"    gear="20"  joint="ankle_x_left"/>
    <motor name="ankle_y_left"    gear="20"  joint="ankle_y_left"/>
    <motor name="shoulder1_right" gear="20"  joint="shoulder1_right"/>
    <motor name="shoulder2_right" gear="20"  joint="shoulder2_right"/>
    <motor name="elbow_right"     gear="40"  joint="elbow_right"/>
    <motor name="shoulder1_left"  gear="20"  joint="shoulder1_left"/>
    <motor name="shoulder2_left"  gear="20"  joint="shoulder2_left"/>
    <motor name="elbow_left"      gear="40"  joint="elbow_left"/>
  </actuator>

  <keyframe>
    <!--
    The values below are split into rows for readibility:
      torso position
      torso orientation
      spinal
      right leg
      left leg
      arms
    -->
    <key name="squat" qpos="0 0 0.596
                            0.988015 0 0.154359 0
                            0 0.4 0
                            -0.25 -0.5 -2.5 -2.65 -0.8 0.56
                            -0.25 -0.5 -2.5 -2.65 -0.8 0.56
                            0 0 0 0 0 0"/>
    <key name="stand_on_left_leg" qpos="0 0 1.21948
                                        0.971588 -0.179973 0.135318 -0.0729076
                                        -0.0516 -0.202 0.23
                                        -0.24 -0.007 -0.34 -1.76 -0.466 -0.0415
                                        -0.08 -0.01 -0.37 -0.685 -0.35 -0.09
                                        0.109 -0.067 -0.7 -0.05 0.12 0.16"/>
  </keyframe>
</mujoco>"""

In [12]:
#@title Humanoid Env

class Humanoid(PipelineEnv):

  def __init__(
      self,
      forward_reward_weight=1.25,
      ctrl_cost_weight=0.1,
      healthy_reward=5.1,
      terminate_when_unhealthy=True,
      healthy_z_range=(1.0, 2.0),
      reset_noise_scale=1e-2,
      exclude_current_positions_from_observation=True,
      inference_fn = None, 
      **kwargs,
  ):
#     path = epath.Path(epath.resource_path('mujoco')) / (
#         'mjx/test_data/humanoid'
#     )
#     mj_model = mujoco.MjModel.from_xml_path(
#         (path / 'humanoid.xml').as_posix())
    mj_model = mujoco.MjModel.from_xml_string(xml_str)
    mj_model.opt.solver = mujoco.mjtSolver.mjSOL_CG
    mj_model.opt.iterations = 6
    mj_model.opt.ls_iterations = 6
    self.mj_model = mj_model

    sys = mjcf.load_model(mj_model)

    physics_steps_per_control_step = 5
    kwargs['n_frames'] = kwargs.get(
        'n_frames', physics_steps_per_control_step)
    kwargs['backend'] = 'mjx'

    super().__init__(sys, **kwargs)

    name2id = lambda x: mujoco.mjx._src.support.name2id(mj_model, mujoco.mjtObj.mjOBJ_GEOM, x)
    
    self.head_id = name2id('head')
    self.foot1_right_id = name2id('foot1_right')
    self.foot1_left_id = name2id('foot1_left')
    self.torso_id = name2id('torso')
    self._forward_reward_weight = forward_reward_weight
    self._ctrl_cost_weight = ctrl_cost_weight
    self._healthy_reward = healthy_reward
    self._terminate_when_unhealthy = terminate_when_unhealthy
    self._healthy_z_range = healthy_z_range
    self._reset_noise_scale = reset_noise_scale
    self._exclude_current_positions_from_observation = (
        exclude_current_positions_from_observation
    )
    
    self.inference_fn = inference_fn

  def reset(self, rng: jp.ndarray) -> State:
    """Resets the environment to an initial state."""
    rng, rng1, rng2 = jax.random.split(rng, 3)
    self.rng = rng
    
    low, hi = -self._reset_noise_scale, self._reset_noise_scale
    qpos = self.sys.qpos0 + jax.random.uniform(
        rng1, (self.sys.nq,), minval=low, maxval=hi
    )
    qvel = jax.random.uniform(
        rng2, (self.sys.nv,), minval=low, maxval=hi
    )
    

    data = self.pipeline_init(qpos, qvel)

    reward, done, zero = jp.zeros(3)
    metrics = {
        'forward_reward': zero,
        'reward_linvel': zero,
        'reward_quadctrl': zero,
        'reward_alive': zero,
        'x_position': zero,
        'y_position': zero,
        'distance_from_origin': zero,
        'x_velocity': zero,
        'y_velocity': zero,
    }
    info = {'counter':zero, 'action':jp.zeros(self.sys.nu), 'rng':rng}

    obs = self._get_obs(data, jp.zeros(self.sys.nu), info['counter'])

    return State(data, obs, reward, done, metrics, info)

  def step(self, state: State, action: jp.ndarray) -> State:
    """Runs one timestep of the environment's dynamics."""

    action = jp.where(
        state.info['counter'] % 200 <= 99,
        action,
        jp.concatenate([
          action[0:3],
          action[9:15],
          action[3:9],
          action[18:21],
          action[15:18]
        ])
    )

    action = action.at[jp.array([15, 17, 18, 20])].set(0)
    
    if self.inference_fn:
        state.info['rng'], act_rng = jax.random.split(state.info['rng'])
        ctrl, _ = self.inference_fn(state.obs, act_rng)
        action = jp.clip(ctrl+action, -1, 1)
        
    data0 = state.pipeline_state
    data = self.pipeline_step(data0, action)

    com_before = data0.subtree_com[self.torso_id]
    com_after = data.subtree_com[self.torso_id]
    velocity = (com_after - com_before) / self.dt
    forward_reward = jp.clip(self._forward_reward_weight * velocity[0], 0, 10)

    min_z, max_z = self._healthy_z_range

    y_head = data.geom_xpos[self.head_id, 2]
    y_mean_feet = (data.geom_xpos[self.foot1_right_id, 2]+ data.geom_xpos[self.foot1_left_id, 2])/2

    done = jp.where(
      (y_head-y_mean_feet) < 0.8,
      1.0,
      0.0
      )

    healthy_reward = self._healthy_reward
    
    ctrl_cost = self._ctrl_cost_weight * jp.sum(jp.square(action))

    state.info['counter'] += 1

    obs = self._get_obs(data, action, state.info['counter'])
    reward = forward_reward + healthy_reward - ctrl_cost

    
    state.metrics.update(
        forward_reward=forward_reward,
        reward_linvel=forward_reward,
        reward_quadctrl=-ctrl_cost,
        reward_alive=healthy_reward,
        x_position=com_after[0],
        y_position=com_after[1],
        distance_from_origin=jp.linalg.norm(com_after),
        x_velocity=velocity[0],
        y_velocity=velocity[1],
    )

    return state.replace(
        pipeline_state=data, obs=obs, reward=reward, done=done, info=state.info
    )

  def _get_obs(
      self, data: mjx.Data, action: jp.ndarray, counter: jp.ndarray
  ) -> jp.ndarray:
    """Observes humanoid body position, velocities, and angles."""
    # external_contact_forces are excluded
    return jp.where(
        counter % 200 <= 99,
        jp.concatenate([
          data.qpos[2:22],
          jp.expand_dims(data.qpos[23], axis=-1),
          jp.expand_dims(data.qpos[26], axis=-1),
          data.qvel[3:21],
          jp.expand_dims(data.qpos[22], axis=-1),
          jp.expand_dims(data.qpos[25], axis=-1),
          action[:15],
          jp.expand_dims(data.qpos[16], axis=-1),
          jp.expand_dims(data.qpos[19], axis=-1),
        ]),
        jp.concatenate([
          data.qpos[2:10],
          data.qpos[16:22],
          data.qpos[10:16],
          jp.expand_dims(data.qpos[26], axis=-1),
          jp.expand_dims(data.qpos[23], axis=-1),
          data.qvel[3:9],
          data.qvel[15:21],
          data.qvel[9:15],
          jp.expand_dims(data.qvel[25], axis=-1),
          jp.expand_dims(data.qvel[22], axis=-1),
          action[0:3],
          action[9:15],
          action[3:9],
          jp.expand_dims(action[19], axis=-1),
          jp.expand_dims(action[16], axis=-1)
        ])
    )


envs.register_environment('humanoid', Humanoid)

In [33]:
def make_inference_fn_from_checkpoint(checkpoint_path,  env):
    cfg = {
        "num_timesteps": 0,
        "num_evals":0,
        "reward_scaling":0.1,
        "episode_length":1000,
        "normalize_observations": True,
        "action_repeat": 1,
        "unroll_length": 10,
        "num_minibatches": 32,
        "num_updates_per_batch": 8,
        "discounting": 0.97,
        "learning_rate": 3e-4,
        "entropy_cost": 1e-3,
        "num_envs": 2048,
        "batch_size": 1024,
        "seed": 0
    }
    
    train_fn = functools.partial(ppo.train, **cfg)

    make_inference_fn, params, _= train_fn(environment=env)

    orbax_checkpointer = ocp.PyTreeCheckpointer()

    checkpoint_params = orbax_checkpointer.restore(checkpoint_path, item=params)
    
    checkpoint_params = (checkpoint_params[0],checkpoint_params[1].policy)
    
    inference_fn = make_inference_fn(checkpoint_params)
    return jax.jit(inference_fn)

In [34]:
env_name = 'humanoid'

inf_fn = make_inference_fn_from_checkpoint(
    "/kaggle/working/227082240",
    envs.get_environment(env_name)
)

env = envs.get_environment(env_name, inference_fn=inf_fn)

/opt/conda/lib/python3.10/site-packages/orbax/checkpoint/type_handlers.py:1544: UserWarning: Couldn't find sharding info under RestoreArgs. Populating sharding info from sharding file. Please note restoration time will be slightly increased due to reading from file instead of directly from RestoreArgs. Note also that this option is unsafe when restoring on a different topology than the checkpoint was saved with.
  warnings.warn(


In [36]:
wandb.init(
    project="tmp",
    config=config
)

## Train Humanoid Policy

In [37]:
import zipfile
import os
def zip_directory(folder_path, zip_path):
    # Ensure the folder exists
    if not os.path.exists(folder_path):
        print(f"The directory {folder_path} does not exist.")
        return

    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file),
                           os.path.relpath(os.path.join(root, file),
                                           os.path.join(folder_path, '..')))
    print(f"Successfully created zip file at {zip_path}")
    
    
    

In [38]:
%notebook notebook.ipynb
wandb.save("/kaggle/working/notebook.ipynb", base_path="/kaggle/working/")

['/kaggle/working/wandb/run-20240905_162847-30rhqtsj/files/notebook.ipynb']

In [ ]:
ckpt_path = epath.Path('/kaggle/working/tmp/humanoid/ckpts')
ckpt_path.mkdir(parents=True, exist_ok=True)

def policy_params_fn(current_step, make_policy, params):
  #save checkpoints
  orbax_checkpointer = ocp.PyTreeCheckpointer()
  save_args = orbax_utils.save_args_from_target(params)
  path = ckpt_path / f'{current_step}'
  orbax_checkpointer.save(path, params, force=True, save_args=save_args)
  zip_path = ckpt_path / f'{current_step}.zip'
  zip_directory(path,zip_path)
  wandb.save(zip_path, base_path="/kaggle/working/")
  

def save_checkpoint(path, params):
  orbax_checkpointer = ocp.PyTreeCheckpointer()
  save_args = orbax_utils.save_args_from_target(params)
  orbax_checkpointer.save(path, params, force=True, save_args=save_args)

train_config = sub_config(config, ['restore_checkpoint_path','num_timesteps', 'num_evals', 'reward_scaling', 'episode_length', 'normalize_observations', 'action_repeat', 'unroll_length', 'num_minibatches', 'num_updates_per_batch', 'discounting', 'learning_rate', 'entropy_cost', 'num_envs', 'batch_size', 'seed'])

# make_networks_factory = functools.partial(
#     ppo_networks.make_ppo_networks,
#         policy_hidden_layer_sizes=(16, 32, 32)
# )

train_fn = functools.partial(ppo.train, **train_config)

times = [datetime.now()]
def progress(num_steps, metrics):
  times.append(datetime.now())
  _metrics = metrics.copy()
  _metrics['numsteps'] = num_steps
  wandb.log(_metrics)

make_inference_fn, params, _= train_fn(environment=env, progress_fn=progress, policy_params_fn=policy_params_fn)

print(f'time to jit: {times[1] - times[0]}')
print(f'time to train: {times[-1] - times[1]}')

/opt/conda/lib/python3.10/site-packages/orbax/checkpoint/type_handlers.py:1544: UserWarning: Couldn't find sharding info under RestoreArgs. Populating sharding info from sharding file. Please note restoration time will be slightly increased due to reading from file instead of directly from RestoreArgs. Note also that this option is unsafe when restoring on a different topology than the checkpoint was saved with.
  warnings.warn(


Successfully created zip file at /kaggle/working/tmp/humanoid/ckpts/11141120.zip


In [ ]:
# import zipfile
# import os
# def zip_directory(folder_path, zip_path):
#     # Ensure the folder exists
#     if not os.path.exists(folder_path):
#         print(f"The directory {folder_path} does not exist.")
#         return

#     with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
#         for root, dirs, files in os.walk(folder_path):
#             for file in files:
#                 zipf.write(os.path.join(root, file),
#                            os.path.relpath(os.path.join(root, file),
#                                            os.path.join(folder_path, '..')))
#     print(f"Successfully created zip file at {zip_path}")

# # Define the folder and zip file paths
# folder_path = "/kaggle/working/tmp/humanoid/ckpts"
# zip_path = "/kaggle/working/ckpts.zip"

# # Call the function to zip the directory
# wandb.init(project='tmp', resume='psw0ok5x')
# save_checkpoint(folder_path, params)
# zip_directory(folder_path, zip_path)
# wandb.save(zip_path, base_path="/kaggle/working/")

In [ ]:
wandb.finish()